In [67]:
import pandas as pd 
import numpy as np
house_sales = "Resources/Perth_Housing_Prices.csv"
sales_df = pd.read_csv(house_sales)
sales_df.set_index('ADDRESS', inplace=True)
sales_df.head()

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [68]:
sales_df['GARAGE'] = sales_df['GARAGE'].fillna(0)

In [69]:
sales_df['SUBURB'].value_counts()

Bertram            231
Iluka              212
Bennett Springs    211
Mindarie           209
Carramar           208
                  ... 
Northbridge         11
Keysbrook            9
O'Connor             8
Neerabup             8
Hope Valley          6
Name: SUBURB, Length: 314, dtype: int64

In [71]:
sales_df.columns

Index(['SUBURB', 'PRICE', 'BEDROOMS', 'BATHROOMS', 'GARAGE', 'LAND_AREA',
       'FLOOR_AREA', 'BUILD_YEAR', 'CBD_DIST', 'NEAREST_STN',
       'NEAREST_STN_DIST', 'DATE_SOLD', 'POSTCODE', 'LATITUDE', 'LONGITUDE',
       'NEAREST_SCH', 'NEAREST_SCH_DIST', 'NEAREST_SCH_RANK'],
      dtype='object')

In [72]:
# column names for recombining dataframe
columns = ['SUBURB', 'PRICE', 'BEDROOMS', 'BATHROOMS', 'GARAGE', 'LAND_AREA',
       'FLOOR_AREA', 'BUILD_YEAR', 'CBD_DIST', 'NEAREST_STN',
       'NEAREST_STN_DIST', 'DATE_SOLD', 'POSTCODE', 'LATITUDE', 'LONGITUDE',
       'NEAREST_SCH', 'NEAREST_SCH_DIST', 'NEAREST_SCH_RANK', 'Prediction']

In [73]:
# create empty dataframe for recombining into
combined_df = pd.DataFrame(columns=columns)
combined_df

,SUBURB,PRICE,BEDROOMS,BATHROOMS,GARAGE,LAND_AREA,FLOOR_AREA,BUILD_YEAR,CBD_DIST,NEAREST_STN,NEAREST_STN_DIST,DATE_SOLD,POSTCODE,LATITUDE,LONGITUDE,NEAREST_SCH,NEAREST_SCH_DIST,NEAREST_SCH_RANK,Prediction


In [74]:
pred_house_sell = {}
temp = {}

# loop on every suburb
for SUBURB in sales_df['SUBURB'].unique():
    # slice
    sales_slice = sales_df[sales_df['SUBURB'] == SUBURB]
    df = sales_slice.copy()

    # set X and y
    y = sales_slice['PRICE']
    X = sales_slice.drop(['SUBURB', 'PRICE', 'NEAREST_STN', 'DATE_SOLD', 'POSTCODE', 'LATITUDE', 'LONGITUDE', 'NEAREST_SCH', 'NEAREST_SCH_RANK', 'BUILD_YEAR', 'CBD_DIST', 'NEAREST_STN_DIST', 'NEAREST_SCH_DIST'], axis=1)
    
    # scale the data
    X_scaled = StandardScaler().fit_transform(X)

    # train, test, split
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y)

    #building linear regression model on training data
    model1= LinearRegression().fit(X_train, y_train)
    pred_house_sell[SUBURB] = model1.predict(X_test)
    
    # make prediction and add it to the sliced dataframe
    prediction = model1.predict(X_scaled)
    df['Prediction'] = prediction
   
    # rebuild full dataframe
    combined_df = pd.concat([combined_df, df], ignore_index=False)

    # print r2 score
    print('Accuracy Score:', r2_score(y_test, pred_house_sell[SUBURB]))  

    # build dictionary with all r2 scores
    temp[SUBURB] = r2_score(y_test, pred_house_sell[SUBURB])

# convert dictionary to dataframe
temp = pd.DataFrame.from_dict(temp, orient ='index').reset_index()

Accuracy Score: 0.20037430664071765
Accuracy Score: 0.39841863090533225
Accuracy Score: 0.06831128812425036
Accuracy Score: 0.47181055654074455
Accuracy Score: 0.27324312395665884
Accuracy Score: 0.5537375356283688
Accuracy Score: 0.3702675808789936
Accuracy Score: 0.20655411825264236
Accuracy Score: 0.7101697893022795
Accuracy Score: 0.21606001600507185
Accuracy Score: 0.45965085406169404
Accuracy Score: 0.27712359371510975
Accuracy Score: -0.74284132395499
Accuracy Score: 0.5573730065476252
Accuracy Score: 0.22835012190331638
Accuracy Score: 0.00502417244185549
Accuracy Score: 0.7545803611263101
Accuracy Score: 0.1909005253721987
Accuracy Score: 0.5265917717329183
Accuracy Score: 0.5324880787109671
Accuracy Score: 0.3725362476501224
Accuracy Score: 0.5451010384894202
Accuracy Score: 0.623209352883285
Accuracy Score: 0.8222825966615694
Accuracy Score: 0.5864189672131861
Accuracy Score: 0.04840935882254749
Accuracy Score: 0.14330650107407583
Accuracy Score: 0.7865694486356648
Accuracy 

In [75]:
# rename columns for merging
temp.columns = ['SUBURB', 'r2 Score']
temp.head()

,SUBURB,r2 Score
0,Floreat,0.200374
1,City Beach,0.398419
2,Dalkeith,0.068311
3,Mosman Park,0.471811
4,Watermans Bay,0.273243


In [76]:
combined_df.head(2)

,SUBURB,PRICE,BEDROOMS,BATHROOMS,GARAGE,LAND_AREA,FLOOR_AREA,BUILD_YEAR,CBD_DIST,NEAREST_STN,NEAREST_STN_DIST,DATE_SOLD,POSTCODE,LATITUDE,LONGITUDE,NEAREST_SCH,NEAREST_SCH_DIST,NEAREST_SCH_RANK,Prediction
35 Oakdale Street,Floreat,2440000,6,4,2,809,442,2009,6800,Loch Street Station,3500,2018-01-08,6014,-31.94060,115.78772,NEWMAN COLLEGE,1.936173,44.0,2.226452e+06
127 Grovedale Road,Floreat,2325000,4,2,2,709,382,2013,6100,Karrakatta Station,3000,2020-01-02,6014,-31.94122,115.79516,NEWMAN COLLEGE,2.135921,44.0,1.799521e+06


In [77]:
# merge r2 Score onto recombined dataframe
sales_df_r2 = pd.merge(combined_df.reset_index(), temp, how='left', on=['SUBURB', 'SUBURB']).set_index('index')
sales_df_r2.head(2)

,SUBURB,PRICE,BEDROOMS,BATHROOMS,GARAGE,LAND_AREA,FLOOR_AREA,BUILD_YEAR,CBD_DIST,NEAREST_STN,NEAREST_STN_DIST,DATE_SOLD,POSTCODE,LATITUDE,LONGITUDE,NEAREST_SCH,NEAREST_SCH_DIST,NEAREST_SCH_RANK,Prediction,r2 Score
index,,,,,,,,,,,,,,,,,,,,
35 Oakdale Street,Floreat,2440000,6,4,2,809,442,2009,6800,Loch Street Station,3500,2018-01-08,6014,-31.94060,115.78772,NEWMAN COLLEGE,1.936173,44.0,2.226452e+06,0.200374
127 Grovedale Road,Floreat,2325000,4,2,2,709,382,2013,6100,Karrakatta Station,3000,2020-01-02,6014,-31.94122,115.79516,NEWMAN COLLEGE,2.135921,44.0,1.799521e+06,0.200374


In [78]:
# remove nan's of r2 scores that didn't have enough data to calculate
sales_df_r2['r2 Score'] = sales_df_r2['r2 Score'].fillna(0)

In [79]:
sales_df_r2.head()

,SUBURB,PRICE,BEDROOMS,BATHROOMS,GARAGE,LAND_AREA,FLOOR_AREA,BUILD_YEAR,CBD_DIST,NEAREST_STN,NEAREST_STN_DIST,DATE_SOLD,POSTCODE,LATITUDE,LONGITUDE,NEAREST_SCH,NEAREST_SCH_DIST,NEAREST_SCH_RANK,Prediction,r2 Score
index,,,,,,,,,,,,,,,,,,,,
35 Oakdale Street,Floreat,2440000,6,4,2,809,442,2009,6800,Loch Street Station,3500,2018-01-08,6014,-31.940600,115.787720,NEWMAN COLLEGE,1.936173,44.0,2.226452e+06,0.200374
127 Grovedale Road,Floreat,2325000,4,2,2,709,382,2013,6100,Karrakatta Station,3000,2020-01-02,6014,-31.941220,115.795160,NEWMAN COLLEGE,2.135921,44.0,1.799521e+06,0.200374
11 Kilkenny Road,Floreat,2300000,4,3,4,809,292,1980,7000,Loch Street Station,4100,2020-01-12,6014,-31.934491,115.787759,NEWMAN COLLEGE,1.256936,44.0,1.723045e+06,0.200374
11 Shannon Street,Floreat,2200000,3,2,2,809,266,2005,7300,Loch Street Station,4000,2020-01-10,6014,-31.936180,115.783780,NEWMAN COLLEGE,1.483140,44.0,1.489595e+06,0.200374
58 Oceanic Drive,Floreat,2200000,5,4,4,809,100,1956,7300,Loch Street Station,3800,2018-01-02,6014,-31.937999,115.783754,NEWMAN COLLEGE,1.681339,44.0,1.301097e+06,0.200374


In [80]:
# write csv to file
sales_df_r2.to_csv('Perth Housing Predictions and r2.csv', sep=',')

In [81]:
temp.to_csv('suburb_r2_scores.csv', sep=',')